In [22]:
import os
import sys
print(os.getcwd())
sys.path.append("../WanPy")

from WanPy import *
from pythtb import *
from pythTB_wan import *
import models
import plotting as plot

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

/Users/treycole/Codes/WanPy/top_obstruction


In [38]:
# tight-binding parameters
delta = 1
t = 1
t2 = -0.4 #-0.1

n_super_cell = 2
model = models.Haldane(delta, t, t2).make_supercell([[n_super_cell, 0], [0, n_super_cell]])
# model.display()

In [39]:
lat_vecs = model.get_lat()
orb_vecs = model.get_orb()
n_orb = n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)

low_E_sites = np.arange(0, n_orb, 2)
high_E_sites = np.arange(1, n_orb, 2)

u_wfs_full = wf_array(model, [20, 20])
u_wfs_full.solve_on_grid([0, 0])
chern = u_wfs_full.berry_flux([i for i in range(n_occ)])/(2*np.pi)

print(f"Low energy sites: {low_E_sites}")
print(f"High energy sites: {high_E_sites}")
print(f"Chern # occupied: {chern: .1f}")

Low energy sites: [0 2 4 6]
High energy sites: [1 3 5 7]
Chern # occupied:  1.0


In [56]:
random = False
low_E = False
omit = True

if random:
    omit_num = 1
    n_tfs = n_occ - omit_num
    tf_list = ["random", n_tfs]
elif omit:
    omit_sites = 2
    tf_list = list(np.setdiff1d(low_E_sites, [omit_sites])) # delta on lower energy sites omitting the last site
    # np.random.choice(low_E_sites, n_tfs, replace=False)
    n_tfs = len(tf_list)
elif low_E:
    tf_list = list(low_E_sites)
    n_tfs = len(tf_list)

Wan_frac = n_tfs/n_occ
print(f"# of Wannier functions: {n_tfs}")
print(f"# of occupied bands: {n_occ}")
print(f"Wannier fraction: {Wan_frac}")

# of Wannier functions: 3
# of occupied bands: 4
Wannier fraction: 0.75


In [57]:
save_name = f'C={chern:.1f}_Delta={delta}_t={t}_t2={t2}'
if random:
    save_name += '_tfxs=rand'
else:
    save_name += f'_tfx={np.array(tf_list, dtype=int)}'

sv_dir = 'data'
if not os.path.exists(sv_dir):
    os.makedirs(sv_dir)
sv_prefix = 'Haldane_sing_vals'

file_name = f"{sv_dir}/{sv_prefix}_{save_name}"
file_name

'data/Haldane_sing_vals_C=1.0_Delta=1_t=1_t2=-0.4_tfx=[0 4 6]'

In [58]:
k_path = [[0, 0], [2/3, 1/3], [.5, .5], [1/3, 2/3], [0, 0]]
label = (r'$\Gamma $',r'$K$', r'$M$', r'$K^\prime$', r'$\Gamma $')
nk = 201
(k_vec, k_dist, k_node) = model.k_path(k_path, nk, report=False)

In [59]:
WF = Wannier(model, [20, 20])
twfs = WF.get_trial_wfs(tf_list)
state_idx = list(range(n_occ))

In [60]:
eigvals, eigvecs = model.solve_all(k_vec, eig_vectors=True)
eigvecs = np.transpose(eigvecs, axes=(1,0,2))

A = WF.tf_overlap_mat(eigvecs, twfs, state_idx)
V, S, Wh = np.linalg.svd(A, full_matrices=False)

np.save(file_name, S)